# CometML

Account is necessary. Login here: https://www.comet.com

Follow the quickstart tutorial: https://www.comet.com/docs/v2/guides/getting-started/quickstart/

In [3]:
pip install comet_ml


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Vic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for twisted>=16.0.0 from https://files.pythonhosted.org/packages/24/83/8d34d264c72d5450cd3d3ef31c885bb36b94635aa8e9e223581793d9d6c8/twisted-23.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for lxml>=2.3 from https://files.pythonhosted.org/packages/5b/d6/e794d996dec76b773691af78390fe2f419ab7cb5b78a4df982e21ae655b7/lxml-5.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for zope.interface>=4.1.1 from https://files.pythonhosted.org/packages/6c/9c/9d3c0e7e5362ea59da3c42b3b2b9fc073db433a0fe3bc6cae0809ccec395/zope.interface-6.1-cp311-cp311-win_amd64.whl.metadata
     ----------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Vic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
## import comet_ml at the top of your file
from comet_ml import Experiment

## Create an experiment with your api key
experiment = Experiment(
    api_key="18kYRUPQaTiwXuPBxACW3coeV",
    project_name="Comet_Example",
    workspace="victors-space",
)

experiment.set_name("data_science_01")


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


COMET INFO: Experiment is live on comet.com https://www.comet.com/victors-space/comet-example/a7d6c473a73c405a86ab418e6860c53a



COMET INFO: Couldn't find a Git repository in 'c:\\Users\\Vic\\Desktop\\Data Scienece' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [7]:
import pandas as pd
from sklearn import set_config
set_config(transform_output="pandas")
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [8]:
# Load the Titanic dataset "Dataset_titanic.csv" from Moodle into a Pandas DataFrame
titanic_df = pd.read_csv("C:/Users/Vic/Desktop/Data Scienece/Datasets-20231016/Dataset_Titanic.csv")

# Save the features 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch' in a variable X 
# and the labels 'Survived' in a variable y
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
X = titanic_df[features]
y = titanic_df['Survived']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Instantiate a Simple Imputer with a Median Strategy
imp = SimpleImputer(strategy="median")

# Instantiate a One Hot Encoder that ignores unknown categories
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Use a Column Transformer to apply the two transformers (Imputer and One-hot-encoder) to the 
# correct columns and pass through all other columns
ct = ColumnTransformer(
    [('ohe', ohe, ['Sex']), 
    ('imputer', imp, ['Age'])],              
    remainder='passthrough'
)

ct.fit_transform(X_train)

# Instantiate rf
clf = RandomForestClassifier(n_estimators=50, random_state=42)

# Create a Pipeline with two steps: preprocessing (Column Transformer) and classifier
pipe = Pipeline([
    ('preprocessor', ct),
    ('classifier', clf)]
)

# Train the Pipeline on the train set
pipe.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = pipe.predict(X_test)

In [9]:
# Print a classification report
print(classification_report(y_test, y_pred))

#print(classification_report(y_test, y_pred, output_dict=True))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84       110
           1       0.75      0.70      0.72        69

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



# Log Data with Comet_ML
https://www.comet.com/docs/v2/guides/getting-started/tutorials/structured-data/

In [10]:
# Helper functions
# ================
def log_classification_report(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    for key, value in report.items():
        if key == "accuracy":
            experiment.log_metric(key, value)
        else:
            experiment.log_metrics(value, prefix=f"{key}")

# specifically log performance on train or test set:
#with experiment.train():
#    log_classification_report(y_train, pipe.predict(X_train))

#with experiment.test():
#    log_classification_report(y_test, pipe.predict(X_test))
                     
def index_to_example(index):
    return X_test.iloc[index, :][["Pclass", "Sex", "Age", "SibSp", "Parch"]].to_json()


In [11]:
# log classification report
log_classification_report(y_test, y_pred)

In [12]:
# log confusion matrix
experiment.log_confusion_matrix(
    y_test.tolist(),
    pipe.predict(X_test).tolist(),
    index_to_example_function=index_to_example,
)

{'web': 'https://www.comet.com/api/asset/download?assetId=878d3ba803d9474d82b6f23fadf3ccaa&experimentKey=a7d6c473a73c405a86ab418e6860c53a',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=878d3ba803d9474d82b6f23fadf3ccaa&experimentKey=a7d6c473a73c405a86ab418e6860c53a',
 'assetId': '878d3ba803d9474d82b6f23fadf3ccaa'}

In [13]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/victors-space/comet-example/a7d6c473a73c405a86ab418e6860c53a
COMET INFO:   Metrics:
COMET INFO:     0_f1-score             : 0.8355555555555555
COMET INFO:     0_precision            : 0.8173913043478261
COMET INFO:     0_recall               : 0.8545454545454545
COMET INFO:     0_support              : 110.0
COMET INFO:     1_f1-score             : 0.7218045112781954
COMET INFO:     1_precision            : 0.75
COMET INFO:     1_recall               : 0.6956521739130435
COMET INFO:     1_support              : 69.0
COMET INFO:     accuracy               : 0.7932960893854749
COMET INFO:     macro avg_f1-score     : 0.7786800334